In [1]:
import cv2
import numpy as np
from mtcnn import MTCNN
import threading

# Load the MTCNN model
detector = MTCNN()

# Load the real and fake input videos
real_video = cv2.VideoCapture('real.mp4')
fake_video = cv2.VideoCapture('fake.mp4')

# Get the video properties
frame_width = int(real_video.get(3) * 2)
frame_height = int(real_video.get(4))
fps = int(real_video.get(cv2.CAP_PROP_FPS))

# Create an output video writer
output_video = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Define a flag to indicate if the videos have ended
end_of_videos = False

# Define a function to process frames
def process_frames():
    global end_of_videos

    while not end_of_videos:
        # Read frames from the real and fake videos
        ret_real, frame_real = real_video.read()
        ret_fake, frame_fake = fake_video.read()

        # If either video reaches its end, set the flag and exit the loop
        if not ret_real or not ret_fake:
            end_of_videos = True
            break

        # Convert the frames to RGB
        frame_real_rgb = cv2.cvtColor(frame_real, cv2.COLOR_BGR2RGB)
        frame_fake_rgb = cv2.cvtColor(frame_fake, cv2.COLOR_BGR2RGB)

        # Detect faces in the real and fake frames
        faces_real = detector.detect_faces(frame_real_rgb)
        faces_fake = detector.detect_faces(frame_fake_rgb)

        # Draw bounding boxes and labels for each face in the real video
        for face in faces_real:
            bbox = face['box']
            x, y, w, h = bbox
            label = 'Real'

            # Draw bounding box and label on the frame
            cv2.rectangle(frame_real, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame_real, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Draw bounding boxes and labels for each face in the fake video
        for face in faces_fake:
            bbox = face['box']
            x, y, w, h = bbox
            label = 'Fake'

            # Draw bounding box and label on the frame
            cv2.rectangle(frame_fake, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame_fake, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Resize the frames to match the output video dimensions
        frame_real = cv2.resize(frame_real, (int(frame_width/2), frame_height))
        frame_fake = cv2.resize(frame_fake, (int(frame_width/2), frame_height))

        # Combine the real and fake frames
        combined_frame = np.concatenate((frame_real, frame_fake), axis=1)

        # Write the combined frame to the output video
        output_video.write(combined_frame)

# Create and start the thread for processing frames
frame_thread = threading.Thread(target=process_frames)
frame_thread.start()

# Wait for the processing thread to finish
frame_thread.join()

# Release the video captures and writer
real_video.release()
fake_video.release()
output_video.release()

print("Output video saved successfully!")

1/1 [==============================] - 0s 67ms/step
Output video saved successfully!
